In [15]:
import torch
from typing import Union, List, Dict, cast, Any
import torch.nn as nn
from PIL import Image
import cv2
from torchinfo import summary
from torchvision.transforms import ToTensor
import polars as pl
from os.path import exists, join
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import sys
sys.path.append("../../")
from VGG.dataset import  VGGDataset
from VGG.model import vgg16

In [9]:
cfgs: Dict[str, List[Union[str, int]]] = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

In [10]:
class VGG(nn.Module):
    def __init__(self, features, num_classes):
        super(VGG, self).__init__()
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((7,7))
        self.classifier = nn.Sequential(
            nn.Linear(512*7*7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [11]:

def make_layers(cfg: List[Union[str, int]], batch_norm: bool = False) -> nn.Sequential:
    layers: List[nn.Module] = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            v = cast(int, v)
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(True)]
            else:
                layers += [conv2d, nn.ReLU(True)]
            in_channels = v
    return nn.Sequential(*layers)

def _vgg(arch: str, cfg: str, batch_norm: bool, **kwargs: Any) -> VGG:
    model = VGG(make_layers(cfg=cfgs[cfg], batch_norm=batch_norm), **kwargs)
    return model

def vgg16(**kwargs: Any)->VGG:
    return _vgg('vgg16', 'D', False, **kwargs)

# data load

In [2]:
df = pl.read_csv("../../datasets/COVID/covid_data.csv")
csv_path= "../../datasets/COVID/covid_data.csv"

In [53]:
class DataCsvParser():
    def __init__(self, csv_path: str):
        import os
        import polars as pl
        if not os.path.exists(csv_path):
            raise FileNotFoundError(f"CSV file not found: {csv_path}")

        self.df = pl.read_csv(csv_path)

    def data_path(self, mode):
        """
        特定のmodeに対応するpath列の値をリストとして返すプロパティ
        """
        try:
            return self.df.filter(pl.col("mode") == mode).get_column("path").to_list()
        except AttributeError:
            return []


In [7]:
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
# データ拡張のためのtransformを定義
transform = A.Compose([
    A.Resize(224,224),  # 64x64にリサイズ
    A.Normalize(mean=(0.1307,), std=(0.3081,)),  # 平均と標準偏差で正規化
    ToTensorV2()  # テンソルに変換
])

dataset = VGGDataset(csv_path=csv_path, mode="train", transform=transform)
dataset[0][0].shape

torch.Size([1, 224, 224])

In [8]:
dataloader = DataLoader(dataset, batch_size=32)

In [13]:
for data in dataloader:
    image, label = data
    break

In [18]:
model = vgg16(num_classes=2)
model(image)

RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[32, 1, 224, 224] to have 3 channels, but got 1 channels instead

In [26]:
from glob import glob
from pathlib import Path
import subprocess
path_list = glob("../../datasets/COVID/images/*")
for path in path_list:
    if Path(path).exists() == False:
        print(path)
    subprocess.run(f"convert {path} -strip {path}")